In [1]:
from model.MF import *
from preprocess.Gowalla import *
from evaluation.MF_evaluation import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/Gowalla'
dataset = Gowalla(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'learning_rate': 1e-5,  # over-fitting
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
    "weight_decay": 1e-7,
    'global_bias':(data.edge_index.size(1) + data.edge_label_index.size(1) + 2) / (num_books * num_users)
}
model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

In [2]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [3]:
config['epochs'] = 100
retrain_model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
retrain_model, recall, ndcg = MF_based_eva(retrain_model, config, retain_data, device)

Epoch 1/100, Train Loss: 0.9978, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 2/100, Train Loss: 0.9959, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 3/100, Train Loss: 0.9940, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 4/100, Train Loss: 0.9921, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 5/100, Train Loss: 0.9902, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 6/100, Train Loss: 0.9884, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 7/100, Train Loss: 0.9865, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 8/100, Train Loss: 0.9846, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 9/100, Train Loss: 0.9828, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 10/100, Train Loss: 0.9809, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0004
Epoch 11/100, Train Loss: 0.9791, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch 12/100, Train Loss: 0.9772, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0004
E

In [4]:
MF_forget_data_eva(retrain_model, None, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0091, Recall@20: 0.0634, NDCG@20: 0.0330


In [5]:
# Define the model
teacher = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
student = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"MF_Gowalla_{config['epochs']}_Epochs_Top_{config['k']}.pt"))
student.load_state_dict(torch.load(f"MF_Gowalla_{config['epochs']}_Epochs_Top_{config['k']}.pt"))

<All keys matched successfully>

In [8]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 50
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_label_index', 'edge_index', 'x'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch 1/50, Train Loss: 0.6986, HR@20: 0.0034, Recall@20: 0.0049, NDCG@20: 0.0055
Epoch 2/50, Train Loss: 0.6986, HR@20: 0.0034, Recall@20: 0.0049, NDCG@20: 0.0055
Epoch 3/50, Train Loss: 0.6986, HR@20: 0.0034, Recall@20: 0.0049, NDCG@20: 0.0055
Epoch 4/50, Train Loss: 0.6985, HR@20: 0.0034, Recall@20: 0.0049, NDCG@20: 0.0054
Epoch 5/50, Train Loss: 0.6985, HR@20: 0.0034, Recall@20: 0.0048, NDCG@20: 0.0054
Epoch 6/50, Train Loss: 0.6985, HR@20: 0.0034, Recall@20: 0.0048, NDCG@20: 0.0054
Epoch 7/50, Train Loss: 0.6985, HR@20: 0.0034, Recall@20: 0.0048, NDCG@20: 0.0054
Epoch 8/50, Train Loss: 0.6985, HR@20: 0.0033, Recall@20: 0.0048, NDCG@20: 0.0054
Epoch 9/50, Train Loss: 0.6985, HR@20: 0.0033, Recall@20: 0.0048, NDCG@20: 0.0054
Epoch 10/50, Train Loss: 0.6985, HR@20: 0.0033, Recall@20: 0.0048, NDCG@20: 0.0053
Epoch 11/50, Train Loss: 0.6985, HR@20: 0.0033, Recall@20: 0.0047, NDCG@20: 0.0053
Epoch 12/50, Train Loss: 0.6985, HR@20: 0.0033, Recall@20: 0.0047, NDCG@20: 0.0053
Epoch 13/50, 

In [9]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0016, Recall@20: 0.0070, NDCG@20: 0.0046


In [ ]:
# Setting the basic hyperparameters
config['Contrastive_loss'] = False
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [12]:
# Setting the basic hyperparameters
config['Contrastive_loss'] = True
config['regularization'] = False

student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

Epoch 1/50, Train Loss: 0.7288, HR@20: 0.0094, Recall@20: 0.0175, NDCG@20: 0.0158
Epoch 2/50, Train Loss: 0.7287, HR@20: 0.0094, Recall@20: 0.0175, NDCG@20: 0.0159
Epoch 3/50, Train Loss: 0.7287, HR@20: 0.0094, Recall@20: 0.0176, NDCG@20: 0.0159
Epoch 4/50, Train Loss: 0.7286, HR@20: 0.0094, Recall@20: 0.0176, NDCG@20: 0.0159
Epoch 5/50, Train Loss: 0.7286, HR@20: 0.0094, Recall@20: 0.0176, NDCG@20: 0.0159
Epoch 6/50, Train Loss: 0.7285, HR@20: 0.0094, Recall@20: 0.0176, NDCG@20: 0.0159
Epoch 7/50, Train Loss: 0.7285, HR@20: 0.0094, Recall@20: 0.0176, NDCG@20: 0.0159
Epoch 8/50, Train Loss: 0.7284, HR@20: 0.0094, Recall@20: 0.0177, NDCG@20: 0.0159
Epoch 9/50, Train Loss: 0.7284, HR@20: 0.0094, Recall@20: 0.0177, NDCG@20: 0.0160
Epoch 10/50, Train Loss: 0.7283, HR@20: 0.0094, Recall@20: 0.0177, NDCG@20: 0.0160
Epoch 11/50, Train Loss: 0.7283, HR@20: 0.0094, Recall@20: 0.0177, NDCG@20: 0.0160
Epoch 12/50, Train Loss: 0.7282, HR@20: 0.0094, Recall@20: 0.0177, NDCG@20: 0.0160
Epoch 13/50, 

In [13]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0042, Recall@20: 0.0219, NDCG@20: 0.0131


In [ ]:
# Setting the basic hyperparameters
config['Contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
config['tuning_type'] = 'gpf-plus'
config["number_p"] = 10 # The number of prompts
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)